# Artificial Neural Network

### Importing the libraries

In [29]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [30]:
tf.__version__

'2.5.0-rc0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [31]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [32]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [33]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [34]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [35]:
print(X[:, 2])

[0 0 0 ... 0 1 0]


One Hot Encoding the "Geography" column

In [36]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [37]:
print(X[1,:])

[0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]


### Splitting the dataset into the Training set and Test set

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=0)

### Feature Scaling

In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [40]:
X_test

array([[-1.00150113,  1.71887259, -0.57176909, ...,  0.64434291,
         0.97774752,  1.6164046 ],
       [ 0.99850112, -0.58177669, -0.57176909, ...,  0.64434291,
        -1.02275892,  0.49757744],
       [-1.00150113, -0.58177669,  1.7489578 , ...,  0.64434291,
         0.97774752, -0.42625077],
       ...,
       [-1.00150113,  1.71887259, -0.57176909, ...,  0.64434291,
         0.97774752, -1.57432676],
       [-1.00150113, -0.58177669,  1.7489578 , ..., -1.55196866,
        -1.02275892,  1.65214902],
       [-1.00150113,  1.71887259, -0.57176909, ..., -1.55196866,
         0.97774752, -1.10217561]])

## Part 2 - Building the ANN

### Initializing the ANN

In [41]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [42]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [43]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [44]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [45]:
ann.compile(optimizer='adam',
           loss='binary_crossentropy',
           metrics=['accuracy'])

### Training the ANN on the Training set

In [46]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
63/63 [==============================] - 0s 547us/step - loss: 0.7067 - accuracy: 0.5118
Epoch 2/100
63/63 [==============================] - 0s 595us/step - loss: 0.6066 - accuracy: 0.7529
Epoch 3/100
63/63 [==============================] - 0s 611us/step - loss: 0.5578 - accuracy: 0.7873
Epoch 4/100
63/63 [==============================] - 0s 627us/step - loss: 0.5275 - accuracy: 0.7809
Epoch 5/100
63/63 [==============================] - 0s 547us/step - loss: 0.4995 - accuracy: 0.7921
Epoch 6/100
63/63 [==============================] - 0s 611us/step - loss: 0.4839 - accuracy: 0.7925
Epoch 7/100
63/63 [==============================] - 0s 579us/step - loss: 0.4635 - accuracy: 0.8027
Epoch 8/100
63/63 [==============================] - 0s 547us/step - loss: 0.4557 - accuracy: 0.8031
Epoch 9/100
63/63 [==============================] - 0s 579us/step - loss: 0.4573 - accuracy: 0.8018
Epoch 10/100
63/63 [==============================] - 0s 740us/step - loss: 0.4503 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [47]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) ## Choosing the Threshold = 0.5

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [48]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [49]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[6079  285]
 [1069  567]]


0.83075